In [2]:
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import PyPDF2
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import faiss

c:\Users\dell\Desktop\MS_DS\M1\S2\AppAutomaique\Projet\loi001_project\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#Converting PDF to TXT
def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    with open('pdf2text.txt', 'w', encoding='utf-8') as file:
        file.write(text)
    return 'pdf2text.txt'

In [5]:
# 1. Load and Embed Documents
pdf2text = extract_text_from_pdf('./loi-n-01-00-portant-organisation-de-lenseignement-supérieur.pdf')
loader = TextLoader('./'+pdf2text) # Replace with your document source
documents = ['pdf2text.txt']

In [11]:
print(torch.__version__)

2.5.0+cpu


loading the tokenizer and the embedding using hugginface models

In [6]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [7]:
embeddings = []
for doc in documents:
    inputs = tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Get the mean pooling of the embeddings (average over tokens)
    embedding = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    embeddings.append(embedding)

embeddings = np.vstack(embeddings)

In [8]:
embedding_dim = embeddings.shape[1]  # Dimension of each embedding
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance index (Euclidean)

# Add the NumPy array of embeddings to the FAISS index
index.add(embeddings)


In [1]:
print(index)


NameError: name 'index' is not defined

In [1]:
query_doc = "Quelles sont les prestations et le financement des services sociaux destinés aux étudiants dans le cadre de la vie universitaire ?"
inputs = tokenizer(query_doc, return_tensors="pt", truncation=True, padding=True)

# Check memory usage here if possible
try:
    query_embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()
    print("Query embedding generated successfully.")
except Exception as e:
    print(f"Error generating embedding: {e}")

NameError: name 'tokenizer' is not defined

In [9]:
# Generate the query embedding (similar to above)
query_doc = "Quelles sont les prestations et le financement des services sociaux destinés aux étudiants dans le cadre de la vie universitaire ?"
inputs = tokenizer(query_doc, return_tensors="pt", truncation=True, padding=True)
query_embedding = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()

# Perform the search
k = 5  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_embedding, k)

# Retrieve results
for i in range(k):
    print(f"Document: {documents[indices[0][i]]}, Distance: {distances[0][i]}")


: 